## Set-up

In [1]:
# Import packages 
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import pandas as pd
import csv
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping#
import keras_tuner

## Dataset splitting and defining target and sample data

In [2]:
# Read in Training dataset
trn = pd.read_csv("smote.csv", header = 0, na_filter = False)
trn

,X,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,class
0,157765.0,1,1.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,5.0,1
1,212853.0,1,1.0,1.0,1.0,35.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,13.0,6.0,6.0,1
2,83531.0,1,1.0,0.0,1.0,41.0,1.0,0.0,1.0,1.0,...,0.0,5.0,0.0,0.0,0.0,1.0,9.0,5.0,4.0,1
3,57522.0,1,1.0,1.0,1.0,41.0,1.0,0.0,0.0,1.0,...,0.0,2.0,3.0,0.0,0.0,0.0,10.0,6.0,8.0,1
4,195039.0,1,1.0,1.0,1.0,31.0,1.0,0.0,1.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430405,253674.0,0,1.0,0.0,1.0,42.0,0.0,0.0,0.0,1.0,...,0.0,3.0,14.0,4.0,0.0,1.0,3.0,6.0,8.0,0
430406,253675.0,0,0.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,6.0,5.0,0
430407,253676.0,0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0,0
430408,253678.0,0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,0


In [3]:
# Check features in dataframe
print(list(trn))

['X', 'Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income', 'class']


In [4]:
# Drop redundant columns 
trn_new = trn.drop(trn.columns[[0]],
                       axis = 1)

In [5]:
# View the cleaned dataframe
trn_new.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,class
0,1,1.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,5.0,1
1,1,1.0,1.0,1.0,35.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,13.0,6.0,6.0,1
2,1,1.0,0.0,1.0,41.0,1.0,0.0,1.0,1.0,1.0,...,0.0,5.0,0.0,0.0,0.0,1.0,9.0,5.0,4.0,1
3,1,1.0,1.0,1.0,41.0,1.0,0.0,0.0,1.0,1.0,...,0.0,2.0,3.0,0.0,0.0,0.0,10.0,6.0,8.0,1
4,1,1.0,1.0,1.0,31.0,1.0,0.0,1.0,1.0,0.0,...,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,7.0,1


In [6]:
# Drop the predictor feature from the dataframe
trn_new2 = trn_new.drop(['Diabetes_binary'], axis = 1)

In [7]:
# View the sample dataframe
trn_new2

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,class
0,1.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,5.0,1
1,1.0,1.0,1.0,35.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,13.0,6.0,6.0,1
2,1.0,0.0,1.0,41.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,5.0,0.0,0.0,0.0,1.0,9.0,5.0,4.0,1
3,1.0,1.0,1.0,41.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,3.0,0.0,0.0,0.0,10.0,6.0,8.0,1
4,1.0,1.0,1.0,31.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430405,1.0,0.0,1.0,42.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,3.0,14.0,4.0,0.0,1.0,3.0,6.0,8.0,0
430406,0.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,6.0,5.0,0
430407,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0,0
430408,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,0


In [8]:
# Drop the predictor feature from the dataframe
trn_new3 = trn_new2.drop(['class'], axis = 1)

In [9]:
trn_new3

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,5.0
1,1.0,1.0,1.0,35.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,13.0,6.0,6.0
2,1.0,0.0,1.0,41.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,5.0,0.0,0.0,0.0,1.0,9.0,5.0,4.0
3,1.0,1.0,1.0,41.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,10.0,6.0,8.0
4,1.0,1.0,1.0,31.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430405,1.0,0.0,1.0,42.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,14.0,4.0,0.0,1.0,3.0,6.0,8.0
430406,0.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,6.0,5.0
430407,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
430408,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0


In [10]:
# Sort the predictor variable
y = trn_new['Diabetes_binary']

In [11]:
# View the dataframe
y

0         1
1         1
2         1
3         1
4         1
         ..
430405    0
430406    0
430407    0
430408    0
430409    0
Name: Diabetes_binary, Length: 430410, dtype: int64

## Early-Stopping

In [12]:
# Creating early stopping
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=20, 
    min_delta=0.001, 
    mode='max'
)

## Static Model

In [13]:
# Define the keras model
model = Sequential()
model.add(Input(shape=(21,)))
model.add(Dense(21, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
# Compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Converting dataframes to tensor
trn_new3 = tf.convert_to_tensor(trn_new3)
y = tf.convert_to_tensor(y)

In [16]:
# Fit the keras model on the dataset
model.fit(trn_new3, y, epochs=1500, batch_size=1500, callbacks = [custom_early_stopping], validation_split=0.2)

Epoch 1/1500
230/230 [==============================] - 1s 3ms/step - loss: 0.6596 - accuracy: 0.6607 - val_loss: 0.7525 - val_accuracy: 0.5736
Epoch 2/1500
230/230 [==============================] - 0s 2ms/step - loss: 0.4841 - accuracy: 0.7823 - val_loss: 0.7299 - val_accuracy: 0.5983
Epoch 3/1500
230/230 [==============================] - 0s 2ms/step - loss: 0.4622 - accuracy: 0.7911 - val_loss: 0.7644 - val_accuracy: 0.5745
Epoch 4/1500
230/230 [==============================] - 1s 3ms/step - loss: 0.4557 - accuracy: 0.7946 - val_loss: 0.7213 - val_accuracy: 0.5996
Epoch 5/1500
230/230 [==============================] - 0s 2ms/step - loss: 0.4520 - accuracy: 0.7968 - val_loss: 0.6734 - val_accuracy: 0.6175
Epoch 6/1500
230/230 [==============================] - 1s 3ms/step - loss: 0.4492 - accuracy: 0.7983 - val_loss: 0.6874 - val_accuracy: 0.6123
Epoch 7/1500
230/230 [==============================] - 1s 3ms/step - loss: 0.4471 - accuracy: 0.7997 - val_loss: 0.6919 - val_accuracy:

230/230 [==============================] - 1s 3ms/step - loss: 0.2754 - accuracy: 0.8726 - val_loss: 0.3119 - val_accuracy: 0.8671
Epoch 114/1500
230/230 [==============================] - 1s 2ms/step - loss: 0.2734 - accuracy: 0.8740 - val_loss: 0.4078 - val_accuracy: 0.7967
Epoch 115/1500
230/230 [==============================] - 1s 2ms/step - loss: 0.2742 - accuracy: 0.8731 - val_loss: 0.3394 - val_accuracy: 0.8388
Epoch 116/1500
230/230 [==============================] - 1s 3ms/step - loss: 0.2725 - accuracy: 0.8737 - val_loss: 0.3790 - val_accuracy: 0.8171
Epoch 117/1500
230/230 [==============================] - 1s 3ms/step - loss: 0.2719 - accuracy: 0.8740 - val_loss: 0.3894 - val_accuracy: 0.8041


## Model function ##

In [17]:
# Creating model function
def get_model(hp):
    model = Sequential()
    model.add(Input(shape=(21,)))
    for i in range(hp.Int("num_layers",1,7)):
        model.add(Dense(units = hp.Int(f"units_{i}",min_value = 10, max_value = 28, step = 3), activation = hp.Choice("activation",["relu","tanh"]),))
    model.add(Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr",min_value = 1e-4,max_value = 1e-2,sampling="log")
    model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate), metrics = ['accuracy'])
    return model

## Tensorboard

In [18]:
# Setting tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs3")

## Keras Tuner

In [19]:
# Defining Keras tuner
tuner = keras_tuner.RandomSearch(hypermodel=get_model,objective="val_accuracy",max_trials=30,executions_per_trial=1,overwrite=True,directory="./benmk/",project_name="diabetes3")

In [20]:
# Whats being captured by the Keras tuner
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 28, 'step': 3, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [21]:
# Fit the keras model on the dataset
tuner.search(trn_new3, y, epochs=1500, batch_size=1500, callbacks = [custom_early_stopping, tensorboard_callback], validation_split=0.2)

Trial 30 Complete [00h 01m 26s]
val_accuracy: 0.8387700319290161

Best val_accuracy So Far: 0.9655096530914307
Total elapsed time: 00h 35m 13s
INFO:tensorflow:Oracle triggered exit


## Tensorboard

In [22]:
# Load the extension for tensorboard
%load_ext tensorboard

In [ ]:
# Reload extension for tensorboard
%reload_ext tensorboard

In [23]:
# Launch tensorboard
%tensorboard --logdir "./logs3"

Reusing TensorBoard on port 6006 (pid 11908), started 3 days, 13:01:21 ago. (Use '!kill 11908' to kill it.)

In [ ]:
# Launch tensorboard alternative way
%tensorboard --logdir "./logs3"  --host localhost